# Contents
 - [Imports](#Imports)
 - [Functions](#Functions)
 - [Make Surface & Visualize](#Surface)
 - [Make Gases & Visualize](#Gases)
 - [Apply Forcefields and Generate Simulation Box](#Forcefields)
 - [Define Statepoints](#Statepoints)
 - [Render Trajectory](#Trajectory)
 - [Errors](#Errors)
 - [Notes](#Notes)

# Interesting Etherpads
 - [7-9: Equilibration & Box Stuff](https://etherpad.boisestate.edu/p/ldrd-s20-9)
 - [7-30: New Sweeps](https://etherpad.boisestate.edu/p/ldrd-s20-12)

In [14]:
# Quick commands
#!tail -n 1 signac/workspace/*/surfequil*.prp
#!head -n 10 signac/workspace/*/surfprod*.prp
!rm -rf *.in.* species* *.out* *.inp* *.log*

<a id="Imports"></a>
## Imports

In [15]:
import mosdef_cassandra as mc
import generate_mc as gen   # Chris' script
import mbuild as mb
import foyer
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
import random
import unyt as u    # For Cassandra's physical unit specification 
                    # https://mosdef-cassandra.readthedocs.io/en/latest/guides/unyts.html

<a id="Functions"></a>
## Functions

In [23]:
def run_cassandra_surf(chem_pot, temp):
    # Create box and species list
    box_list = [packed]
    species_list = [typed_packed,typed_ethane] #

    # Since we have an occupied box we need to specify
    # the number of each species present in the intial config
    mols_in_boxes = [[1,0]]

    system = mc.System(box_list, species_list, mols_in_boxes=mols_in_boxes)
    moveset = mc.MoveSet("gcmc", species_list)

    custom_args = {
        "run_name": f"surfequil",
        "chemical_potentials": ["none",chem_pot*u.Unit('kJ/mol')],
        "rcut_min": 0.3980 * 2.5* u.angstrom, #(or 3.0)
        "vdw_cutoff": min(box.lengths)/2.1* u.angstrom,
        "charge_style": "none",
        #"charge_cutoff": 14.0,
        "coord_freq": 100,
        "prop_freq": 10,
    }

    mc.run(
        system=system, 
        moveset=moveset, 
        run_type="equilibration", 
        run_length= 10000, # To reach ~1.33 hours
        temperature=temp*u.K, 
        **custom_args
    )
    
    # Set max translate and volume for production
    moveset.max_translate = [[0* u.angstrom,10.0* u.angstrom]] # angstroms

    # Update run_name and restart_name
    custom_args["run_name"] = f"surfprod_{chem_pot:.0f}_{temp:.0f}"
    custom_args["restart_name"] = f"surfequil"

    mc.restart(
        system=system,
        moveset=moveset,
        run_type="production",
        run_length= 50000, # To reach ~6.67 hours, total 8 hrs
        temperature=temp*u.K,
        **custom_args,
    )

In [17]:
mc.utils.detect_cassandra_binaries();

Using the following executables for Cassandra:
Python: /home/nealeellyson/miniconda3/envs/moscas/bin/python
library_setup: /home/nealeellyson/miniconda3/envs/moscas/bin/library_setup.py
Cassandra: /home/nealeellyson/miniconda3/envs/moscas/bin/cassandra.exe


<a id="Surface"></a>
## Make Surface & Visualize

In [18]:
packed = mb.load("/home/erjank_project/nealeellyson/LDRD/rhaco-mc/surface-gen/1x1x2-benzene-nonbonded.mol2")
packed.translate_to(np.zeros(3)) # Added so that periodicity doesn't error out
packed.periodicity = packed.boundingbox.lengths # necessary
#packed.visualize().show()

<a id="Gases"></a>
## Make Gases & Visualize

In [19]:
ethane = mb.load("CC", smiles=True)
#ethane.visualize().show()

<a id="Forcefields"></a>
## Apply Forcefields and Generate Simulation Box

In [20]:
# Load forcefields
#opls_uff = foyer.forcefields.Forcefield(forcefield_files='forcefields/FF_opls_uff.xml')
oplsaa = foyer.forcefields.load_OPLSAA()
M1oplsaa_nonB = foyer.forcefields.Forcefield(forcefield_files='forcefields/M1_oplsaa-nonB.xml')
M1oplsaa = foyer.forcefields.Forcefield(forcefield_files='forcefields/M1_oplsaa.xml')

# Use foyer to apply forcefields
typed_ethane = M1oplsaa.apply(ethane) 
typed_packed = M1oplsaa_nonB.apply(packed,
                            assert_angle_params=False,
                            assert_bond_params=False,
                            assert_dihedral_params=False,
                            assert_improper_params=False)

# Establish simulation box dimensions
L = 10
box = mb.Box(mins=[-L/2]*3, maxs=[L/2]*3)

/home/nealeellyson/miniconda3/envs/moscas/lib/python3.7/site-packages/parmed/openmm/topsystem.py:238: OpenMMWarning: Adding what seems to be Urey-Bradley terms before Angles. This is unexpected, but the parameters will all be present in one form or another.
  'all be present in one form or another.', OpenMMWarning)


<a id="Statepoints"></a>
## Define Statepoints

In [21]:
eps = typed_ethane.atoms[0].epsilon
print(f"epsilon: {eps:.2f}")
print("sigma: ", typed_ethane.atoms[0].sigma)
# ParmEd is in kcal/mol

kb = 0.001985875    # kcal/(mol K)
T = eps/kb

print(f"The unit temperature is {T:.0f} K")
chem_pots = [-10] #[i*T for i in range(0,10)] 
temp = 800

epsilon: 0.07
sigma:  3.5
The unit temperature is 33 K


<a id="Run"></a>
## Run Simulation

In [22]:
for chem_pot in chem_pots:
    run_cassandra_surf(chem_pot,temp)

Treating <Structure 416 atoms; 1 residues; 0 bonds; PBC (orthogonal); NOT parametrized> as a non-insertable rigid species since it has no bonds
Using the following executables for Cassandra:
Python: /home/nealeellyson/miniconda3/envs/moscas/bin/python
library_setup: /home/nealeellyson/miniconda3/envs/moscas/bin/library_setup.py
Cassandra: /home/nealeellyson/miniconda3/envs/moscas/bin/cassandra.exe
Generating fragment libraries...
Running Cassandra...


CassandraRuntimeError: Cassandra exited with an error, see mosdef_cassandra_2020-08-19_13:16:45.731968.log for details.

<a id="Trajectory"></a>
## Render Trajectory

In [27]:
import ex_render_cassandra as erc
import os
import getpass

# These cells are for visualizing output
xyzfile = f"prod_{chem_pot:.0f}_{temp:.0f}.out.xyz"
boxfile = f"prod_{chem_pot:.0f}_{temp:.0f}.out.H"
traj = erc.make_traj(xyzfile, boxfile)
print("Frames: ",len(traj))

cwd = os.getcwd()
username = getpass.getuser()
for chem_pot in chem_pots:
    xyzfile = f"surfprod_{chem_pot:.0f}_{temp:.0f}.out.xyz"
    print("\n\tCopy this into Ovito Load Remote File:") # From Fry
    print('sftp://'+username+'@fry'+cwd+'/'+xyzfile)

Copy this into Ovito Load Remote File:
/home/nealeellyson/CassandraSim/output/gcmc.out.xyz


In [26]:
xyzfile = "output/gcmc.out.xyz"
boxfile = "output/gcmc.out.H"

traj = erc.make_traj(xyzfile, boxfile)

In [27]:
erc.display_movie(erc.render_sphere_frame, traj)
#erc.display_movie(erc.render_sphere_frame, traj, gif="traj.gif")

<a id="Errors"></a>
## Errors:
```
FoyerError: Found no types for atom 320 (6).
```
The offending atom was traced to be the carbons/hydrogens in the rigid benzenes coating the surface. Being a Foyer issue, it was determined that the current forcefield `M1oplsaa.xml` did not have parameters for unbonded benzene atoms. This was resolved by creating a simple pdb file of a couple of benzenes in Avogadro (SMILES strings could also be used) and sending it into the `AtomTyping.ipynb` and generating `smaller.xml` which shows the oplsaa atom types and those can be altered in a copy of the `M1oplsaa.xml`. 

In `smaller.xml`:
```
<ForceField>
  <AtomTypes>
    <Type name="opls_145" class="CA" element="C" mass="12.01078" def="[C;X3;r6]1[C;X3;r6][C;X3;r6][C;X3;r6][C;X3;r6][C;X3;r6]1" desc="" doi="10.1021/ja9621760" overrides="">
      <!--Note: original overrides="opls_141,opls_142"-->
    </Type>
    <Type name="opls_146" class="HA" element="H" mass="1.007947" def="[H][C;%opls_145]" desc="" doi="10.1021/ja9621760" overrides="">
      <!--Note: original overrides="opls_144"-->
    </Type>
  </AtomTypes>
  <NonbondedForce coulomb14scale="0.5" lj14scale="0.5">
    <Atom type="opls_145" charge="-0.115" sigma="0.355" epsilon="0.29288"/>
    <Atom type="opls_146" charge="0.115" sigma="0.242" epsilon="0.12552"/>
  </NonbondedForce>
  <HarmonicBondForce>
    <Bond type1="opls_145" type2="opls_145" length="0.14" k="392459.2"/>
    <Bond type1="opls_145" type2="opls_146" length="0.108" k="307105.6"/>
  </HarmonicBondForce>
  <HarmonicAngleForce>
    <Angle type1="opls_145" type2="opls_145" type3="opls_145" angle="2.0943951024" k="527.184"/>
    <Angle type1="opls_145" type2="opls_145" type3="opls_146" angle="2.0943951024" k="292.88"/>
  </HarmonicAngleForce>
  <RBTorsionForce>
    <Proper type1="opls_145" type2="opls_145" type3="opls_145" type4="opls_145" c0="30.334" c1="0.0" c2="-30.334" c3="0.0" c4="0.0" c5="0.0"/>
    <Proper type1="opls_145" type2="opls_145" type3="opls_145" type4="opls_146" c0="30.334" c1="0.0" c2="-30.334" c3="0.0" c4="0.0" c5="0.0"/>
    <Proper type1="opls_146" type2="opls_145" type3="opls_145" type4="opls_146" c0="30.334" c1="0.0" c2="-30.334" c3="0.0" c4="0.0" c5="0.0"/>
  </RBTorsionForce>
</ForceField>
```

Where `opls_145` and `opls_146` need to be revised in the `M1oplsaa.xml` copy to have 
```
  <Type name="opls_145" class="CA" element="C" mass="12.01100" def="[C]" overrides="opls_141,opls_142" doi="10.1021/ja9621760"/>
  <Type name="opls_146" class="HA" element="H" mass="1.00800"  def="[H]" overrides="opls_144" desc="benzene H" doi="10.1021/ja9621760"/>
```
Where the only change is in `def=[C]`. The forcefield can then be applied to a packed, non-bonded surface.  


```
Atomic overlap in the configuration
This error occurred in subroutine Check_System_Energy on step 0.
```



<a id="Notes"></a>
## Notes